# Homework 7
12110418 庄子鲲
### Problem 1
Using Python and Numpy, write a class named DecisionTree, which implements the decision tree model based on information gain.

1. Calculate the information entropy:
$$
H(Y|X=x)=-\sum_{y\in{Y}}{P(y|x)\log_{2}{P(y|x)}}
$$
2. Calculate the imformation gain:
$$
IG(Y)=H(Y)-H(Y|X)
$$
3. Choose the feature that has maximum IG as the root node
4. Continue calculate the information gain of the rest features and choose the one with the largest IG.
5. If the output are the same, stop gorwing that branch.
6. Complete the decision tree.

In [20]:
import numpy as np
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._grow_tree(X, y)

    def _calculate_entropy(self, y):
        unique_classes, class_counts = np.unique(y, return_counts=True)
        probabilities = class_counts / len(y)
        entropy = -np.sum(probabilities * np.log2(probabilities))
        return entropy

    def _calculate_information_gain(self, X, y, feature_index):
        unique_features, featrue_counts = np.unique(X[:, feature_index],return_counts=True)
        mask=[]
        for i in range(len(unique_features)):
            mask.append(X[:, feature_index] == unique_features[i])
        # Calculate entropy for the two subsets
        entropy_parent = self._calculate_entropy(y)
        entropy_leaves=[]
        for i in range(len(mask)):
            entropy_leaves.append(self._calculate_entropy(y[mask[i]]))
        # Calculate information gain
        size=[]
        total_size=0
        for i in range(len(mask)):
            size.append(np.sum(mask[i]))
            total_size+=np.sum(mask[i])

        information_gain = entropy_parent
        
        for i in range(len(mask)):
            information_gain -= (size[i] / total_size) * entropy_leaves[i]
        return information_gain
    
    # Find the best feature to split on
    def _find_best_split(self, X, y):
        num_samples, num_features = X.shape
        best_information_gain = -1
        best_split = None
        
        for feature_index in range(num_features):
            information_gain = self._calculate_information_gain(X, y, feature_index)
            if information_gain > best_information_gain:
                best_information_gain = information_gain
                best_split = feature_index
        return best_split

    def _grow_tree(self, X, y, depth=0):
        # Check termination conditions
        if depth == self.max_depth or np.all(y == y[0]):
            return {"class": np.argmax(np.bincount(y)), "depth": depth}

        # Find the best feature and split on it
        best_split = self._find_best_split(X, y)
        if best_split is None:
            return {"class": np.argmax(np.bincount(y)), "depth": depth}
        
        feature_index = best_split
        mask=[]
        unique_features, featrue_counts = np.unique(X[:, feature_index],return_counts=True)
        for i in range(len(unique_features)):
            mask.append(X[:, feature_index] == unique_features[i])
            
        # Grow the subtrees
        subtree=[]
        for i in range(len(mask)):
            subtree.append(self._grow_tree(X[mask[i]], y[mask[i]], depth + 1))
        
        return{
            "feature_index": feature_index,
            "values":unique_features,
            "subtree": subtree,
            "depth": depth,
        }   


### Problem 2:
Consider the dataset lenses.data (the link is also provided) and use the DecisionTree class to build a decsion tree for this dataset.

In [21]:
# Load Data and Grow the Decision Tree
lenses=np.loadtxt('lenses.data',usecols=(1,2,3,4,5))
lenses = lenses.astype('int64', casting='unsafe')
X_train=lenses[:, :4]
y_train=lenses[:,-1]
model = DecisionTree(max_depth=None)
model.fit(X_train, y_train)
# Print the decision tree
def print_decision_tree_array(tree_dict, values=None, indent=0):
    """
    Recursively print the decision tree dictionary with proper indentation.
    """
    # Base case: leaf node
    if 'class' in tree_dict:
        print(f"{'  ' * indent}Output: {tree_dict['class']}")
        return
    
    # Non-leaf node
    values_str = f"{tree_dict['feature_index']}, values: {np.array2string(tree_dict['values'])}"
    print(f"{'  ' * indent}Feature index: {values_str}")
    
    # Print subtrees
    for i, subtree in enumerate(tree_dict['subtree']):
        value = tree_dict['values'][i] if 'values' in tree_dict else None
        value_str = f"value {value}" if value is not None else ""
        print(f"{'  ' * (indent + 1)}{value_str}:", end=" ")
        print_decision_tree_array(subtree, values + [value] if values else [value], indent + 1)
print("Decision Tree: ")
print_decision_tree_array(model.tree)

Decision Tree: 
Feature index: 3, values: [1 2]
  value 1:   Output: 3
  value 2:   Feature index: 2, values: [1 2]
    value 1:     Feature index: 0, values: [1 2 3]
      value 1:       Output: 2
      value 2:       Output: 2
      value 3:       Feature index: 1, values: [1 2]
        value 1:         Output: 3
        value 2:         Output: 2
    value 2:     Feature index: 1, values: [1 2]
      value 1:       Output: 1
      value 2:       Feature index: 0, values: [1 2 3]
        value 1:         Output: 1
        value 2:         Output: 3
        value 3:         Output: 3
